Primero vamos a conectarnos con el servidor de origen y listar sus proyectos

In [ ]:
from redminelib import Redmine
import cfg_src_rm

if cfg_src_rm.ignore_cert:
    src_rm = Redmine(cfg_src_rm.server_url,key=cfg_src_rm.api_key, requests={'verify': False})
else:
    src_rm = Redmine(cfg_src_rm.server_url,key=cfg_src_rm.api_key)

src_prjs = src_rm.project.all()

print("Proyectos del servidor origen:")
for p in src_prjs:
    print ("\t",p.identifier," \t| ",p.name)


Ahora vamos a averiguar los identificadores de los campos personalizados que necesitaremos, mientras construimos un diccionario de ellos:

In [ ]:
src_cfs = {}
for cf in src_rm.custom_field.all():
    print(cf)
    src_cfs[cf.name] = cf
    
print(src_cfs.keys())

Listaremos las tareas mientras construimos un diccionario para saber encontrar las tareas por subject

In [ ]:
src_prj = src_rm.project.get(cfg_src_rm.project_id)
print ("Obtenemos proyecto: ",src_prj.identifier," | ",src_prj.name)

src_prj_issues = src_rm.issue.filter(
    project_id=src_prj.id,
    sort='subject:asc',
    status_id='*'
)

src_tsks = {}
src_tsks_id = {}
for i in src_prj_issues:    
    print(i.subject)
    src_tsks[i.subject] = src_rm.issue.get(i.id)
    src_tsks_id[i.id] = src_rm.issue.get(i.id)
    
    # Hemos de obtener el título
    #print("cfsid:",src_cfs['WrkloadPct'].id)
    valid_cv = False
    cv = i.custom_fields.get(src_cfs['IssTitle'].id)
    thistitle = "NO TITLE"
    if cv is not None:
        if hasattr(cv,'value'):
            cv_val = cv.value
            if(cv_val != ""):
                print("title:",cv_val)
                thistitle = cv_val
                
    src_tsks[i.subject]['title'] = thistitle
    src_tsks_id[i.id]['title'] = thistitle
        
    # Hemos de obtener el workload
    valid_cv = False
    cv = i.custom_fields.get(src_cfs['WrkloadPct'].id)
    wload = 0
    wloadstr = "*0*"
    if cv is not None:
        if hasattr(cv,'value'):
            cv_val = cv.value
            if(cv_val != ""):
                print("wloadstr:",cv_val)
                wload = int(cv_val)
                wloadstr = cv_val                

    src_tsks[i.subject]['wload'] = wload
    src_tsks_id[i.id]['wload'] = wload    
    src_tsks[i.subject]['wloadstr'] = wloadstr
    src_tsks_id[i.id]['wloadstr'] = wloadstr

    print(i.id)
    
print(src_tsks.keys())

In [ ]:
print("Source Redmine users")
src_users_login = {}
src_users_id = {}
for u in src_rm.user.all():
    src_users_login[u.login] = u
    src_users_id[u.id] = u
    src_users_id[u.id]['tsks'] = []
    print(src_users_id[u.id]['tsks'])
    print(src_users_id[u.id])

print(src_users_login.keys())

print("Source Redmine groups")
src_groups_login = {}
src_groups_id = {}
for u in src_rm.group.all():
    src_groups_login[u.name] = u
    src_groups_id[u.id] = u
    

print(src_groups_login.keys())


Ahora comprobamos los status de uno y otro lado

In [ ]:
print("Source Redmine statuses")
src_statuses = {}
src_statuses_id = {}
for st in src_rm.issue_status.all():
    src_statuses[st.name] = st
    src_statuses_id[st.id] = st
    
print(src_statuses.keys())


Ahora comprobamos las prioridades de uno y otro lado

In [ ]:
print("Source Redmine priorities")
src_priorities = {}
src_priorities_id = {}
for prio in src_rm.enumeration.filter(resource='issue_priorities'):
    src_priorities[prio.name] = prio
    src_priorities_id[prio.id] = prio
    
print(src_priorities.keys())


In [ ]:
for tsk in src_tsks.keys():
    if (hasattr(src_tsks[tsk],'assigned_to')):
        assig = src_tsks[tsk].assigned_to
        print(assig)
        print(assig.id)
        if assig.id in src_users_id.keys():
            print(src_users_id[assig.id].login)
            src_users_id[assig.id]['tsks'] += [src_tsks[tsk]]
            
        else:
            if assig.id in src_groups_id.keys():
                print(src_groups_id[assig.id].name)
            else:
                print("No encontramos a ",assig.id)
            

In [ ]:
for k in src_users_id.keys():
    u=src_users_id[k]
    print(u.login)
    print(u['tsks'])

Mostramos las tareas de los usuarios de hoy

In [20]:
import datetime
from pyexcel_ods import save_data
from collections import OrderedDict

import odswriter as ods
import decimal

def date_by_adding_business_days(from_date, add_days):
    business_days_to_add = add_days
    current_date = from_date
    while business_days_to_add > 0:
        current_date += datetime.timedelta(days=1)
        weekday = current_date.weekday()
        if weekday >= 5: # sunday = 6
            continue
        business_days_to_add -= 1
    return current_date



'''
    odsfile.writerow(["String", "ABCDEF123456", "123456"])
    # Lose the 2L below if you want to run this example code on Python 3, Python 3 has no long type.
    odsfile.writerow(["Float", 1, 123, 123.123, decimal.Decimal("10.321")])
    odsfile.writerow(["Date/DateTime", datetime.datetime.now(), datetime.date(1989, 11, 9)])
    odsfile.writerow(["Time",datetime.time(13, 37),datetime.time(16, 17, 18)])
    odsfile.writerow(["Bool", True, False, True])
    odsfile.writerow(["Formula", 1, 2, 3, ods.Formula("IF(A1=2,B1,C1)")])
'''


# Open odswriter spreadsheet
with ods.writer(open("file2.ods","wb")) as odsfile:
    data = OrderedDict() # from collections import OrderedDict

    for k in src_users_id.keys():
        u=src_users_id[k]
        usersheet = odsfile.new_sheet(u.login)
        print("\n\n********",u.login)

        initdate = datetime.date.today()
        thisdate = datetime.date.today()
        dayscounter = 0
        header_row = ["ID","Title","Start date","Due date","WloadAcum"]
        subheader_row = ["","","","","Totals","Use this row for your totalling formulas"]
        subheader_row2 = ["","","","","Totals"]
        while dayscounter <= 20:
            dayscounter += 1    
            thisdate = date_by_adding_business_days(initdate, dayscounter)
            header_row += [str(thisdate)]
            subheader_row2 += [ods.Formula("=SUM(INDIRECT(ADDRESS(ROW()+1;COLUMN();1;1)):INDIRECT(ADDRESS(ROW()+500;COLUMN();1;1)))")] 

        sheet = [header_row]+[subheader_row]
        usersheet.writerow(header_row)
        usersheet.writerow(subheader_row2)

        for t in u['tsks']:
            if (hasattr(t,'start_date')):
                tstartdate = t.start_date
            else:
                tstartdate = None

            if (hasattr(t,'due_date')):
                tduedate = t.due_date
            else:
                tduedate = None


            wloadAcum = 0
            thisdate = datetime.date.today()
            dayscounter = 0
            thisrowheader = [t.subject,t['title'],str(tstartdate),str(tduedate)]
            thisrow = []
            thisrow2 = []
            print(thisrowheader)
            while dayscounter <= 20:
                print("Tratamos fecha:",thisdate)
                ok_start =  (tstartdate is None or tstartdate <= thisdate)
                ok_due =  (tduedate is None or tduedate >= thisdate)
                #print(t.subject,ok_start,ok_due)
                if (ok_start and ok_due):
                    # Nos toca contabilizar el workload
                    wloadAcum += t['wload']
                    thisrow += [t['wloadstr']]
                    thisrow2 += [t['wload']]

                    print("\nTotal para",u.login,"para",thisdate,wloadAcum)

                else:
                    thisrow += [""]
                    thisrow2 + [""]

                dayscounter += 1
                thisdate = date_by_adding_business_days(initdate, dayscounter)

            if wloadAcum > 0:
                thisrowheader += [wloadAcum]
            else:
                thisrowheader += [""]

            sheet += [thisrowheader + thisrow]
            usersheet.writerow(thisrowheader + thisrow2)

        data.update({u.login: sheet})

    save_data("your_file.ods", data)



******** admin


******** aescobar


******** amg


******** amm
['mcGRS-0002', 'Especificación montura RED 854+1032', '2019-06-17', '2019-06-28']
Tratamos fecha: 2020-10-28
Tratamos fecha: 2020-10-29
Tratamos fecha: 2020-10-30
Tratamos fecha: 2020-11-02
Tratamos fecha: 2020-11-03
Tratamos fecha: 2020-11-04
Tratamos fecha: 2020-11-05
Tratamos fecha: 2020-11-06
Tratamos fecha: 2020-11-09
Tratamos fecha: 2020-11-10
Tratamos fecha: 2020-11-11
Tratamos fecha: 2020-11-12
Tratamos fecha: 2020-11-13
Tratamos fecha: 2020-11-16
Tratamos fecha: 2020-11-17
Tratamos fecha: 2020-11-18
Tratamos fecha: 2020-11-19
Tratamos fecha: 2020-11-20
Tratamos fecha: 2020-11-23
Tratamos fecha: 2020-11-24
Tratamos fecha: 2020-11-25
['mcGRS-0003', 'Especificación montura RED GREGOR', '2019-10-01', '2019-10-03']
Tratamos fecha: 2020-10-28
Tratamos fecha: 2020-10-29
Tratamos fecha: 2020-10-30
Tratamos fecha: 2020-11-02
Tratamos fecha: 2020-11-03
Tratamos fecha: 2020-11-04
Tratamos fecha: 2020-11-05
Tratamos fecha:

In [18]:
print("hecho")

hecho
